In [5]:
from dsn_ae import DSNAE
from mlp import MLP
import os
import torch
from encoder_decoder import  EncoderDecoder_1

dir_path = "/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/"
# model_save_folder = "/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/pretrain_num_epochs_0_train_num_epochs_100_dop_0.0"
model_save_folder = os.path.join(dir_path,os.listdir(dir_path,)[0])
print(model_save_folder)
input_dim = 1672 #for BRCA

# Pretrain model------
shared_encoder = MLP(input_dim=input_dim,
                         output_dim=128,
                         hidden_dims=[512, 256],
                         dop=0.1).to("cuda")

shared_decoder = MLP(input_dim=2 * 128,
                         output_dim=input_dim,
                         hidden_dims=[512, 256][::-1],
                         dop=0.1).to("cuda")
t_dsnae = DSNAE(shared_encoder=shared_encoder,
                    decoder=shared_decoder,
                    alpha=1,
                    input_dim=input_dim,
                    latent_dim=128,
                    hidden_dims=[512, 256],
                    dop=0.1).to("cuda")
t_dsnae.load_state_dict(torch.load(os.path.join(model_save_folder, 'a_t_dsnae.pt')))


# Finetune model------
fold_count = 0

ft_encoder = t_dsnae.shared_encoder
target_decoder = MLP(input_dim=128 + 300,
                         output_dim=1,
                         hidden_dims=[64, 32]).to("cuda")
target_classifier = EncoderDecoder_1(encoder=ft_encoder, decoder=target_decoder, 
                                        normalize_flag=True).to("cuda")
target_classifier.load_state_dict(
        torch.load(os.path.join(model_save_folder, f'target_classifier_{fold_count}.pt')))

# Print
net = target_classifier
for name,parameters in net.named_parameters():
        print(name)
        print(parameters)

/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/pretrain_num_epochs_300_train_num_epochs_1500_dop_0.1
encoder.module.0.0.weight
Parameter containing:
tensor([[ 0.0040, -0.0119,  0.0221,  ..., -0.0170,  0.0311,  0.0239],
        [ 0.0086, -0.0048,  0.0055,  ..., -0.0059,  0.0132, -0.0124],
        [ 0.0151,  0.0051,  0.0230,  ..., -0.0265,  0.0063, -0.0009],
        ...,
        [ 0.0222,  0.0292,  0.0055,  ...,  0.0094,  0.0012,  0.0186],
        [ 0.0267, -0.0132,  0.0116,  ...,  0.0145,  0.0027, -0.0100],
        [-0.0030, -0.0004, -0.0330,  ...,  0.0075,  0.0112, -0.0091]],
       device='cuda:0', requires_grad=True)
encoder.module.0.0.bias
Parameter containing:
tensor([-3.0790e-02, -2.0689e-03,  4.2055e-03, -4.6941e-02,  1.2451e-02,
        -3.4437e-02,  7.2106e-03, -2.3709e-02, -1.0553e-02, -2.1470e-02,
         4.2521e-02,  1.1770e-04,  4.7789e-02, -6.5795e-03, -5.3668e-02,
        -2.1119e-02, -2.3066e-02, -3.1425e-03, -1.4693e-02,  2.9198e-02,


In [17]:
import pandas as pd
import numpy as np
import time 
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                     index=['K0', 'K1', 'K2'])    
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                      index=['K0', 'K1', 'K2'])
start=time.time()
result = left.index.join(right)              #默认how='left' 
end=time.time()
print('Running time1: %s Seconds'%(end-start))

start=time.time()
result = pd.DataFrame(data=right, index=['K2',"K1","K2","K0"])
# start=time.time()
# result = left.loc[right.index.values]
end=time.time()
print('Running time2: %s Seconds'%(end-start))

result

# right = right.to_numpy()
right.index.values[[1,0,2]]

Running time1: 0.0010607242584228516 Seconds
Running time2: 0.0015752315521240234 Seconds


array(['K1', 'K0', 'K2'], dtype=object)

In [15]:
import pandas as pd
drug_emb = pd.read_csv("../data/preprocessed_dat/drug_embedding/CCL_dataset/supervised_infomax.pth300_results.csv",index_col=1)
drug_emb = drug_emb.iloc[:,1:]
# drug_emb.loc["CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C1)C2=O)NCC=C)C)OC)OC(=O)N)C)C)O)OC"]
len(drug_emb.index.values)

233

: 

In [81]:
import pandas as pd
import numpy as np
import json

a_dict = {
    't1': [7.2,4],
    't2': [4,8],
    't3': [2,6]
}
a = pd.json_normalize(a_dict).T
a.to_csv("../a.csv")
np.argsort(a.iloc[:,0].map(np.mean).values)[::-1][:2]
# for i in range(a.shape[1]):
#     print(i)
#     print(a.iloc[:,i].values)
#     for j in a.iloc[:,i].values:
#         print(j)

# judge_metrics = pd.read_csv("/home/wuys/pdr2/results/all_test/dsrn_adv_norm/brca/raw/Percent_sd/all_CCL/raw/BRCA/gdsc1_raw/all/judge_metrics_results.csv",index_col=0)
# print(judge_metrics.iloc[:,0])
# print(a.iloc[:,0].dtype)
with open("/home/wuys/pdr2/results/all_test/dsrn_adv_norm/brca/raw/Percent_sd/all_CCL/raw/BRCA/gdsc1_raw/all/judge_metrics_results.json",'r') as f:
    judge_metrics = json.load(f)
judge_metrics = pd.json_normalize(judge_metrics).T
top_n = 5
top_n_index = np.argsort(judge_metrics.iloc[:,0].map(np.mean).values)[::-1][:top_n]
judge_metrics
top_n_index



array([1, 0])

In [87]:
import itertools
params_grid = {
            "pretrain_num_epochs": [0, 100, 300],
            "train_num_epochs": [100, 200, 300, 500, 750, 1000, 1500, 2000, 2500, 3000],
            "dop": [0.0, 0.1]
        }
keys, values = zip(*params_grid.items())
update_params_dict_list = [dict(zip(keys, v)) for v in itertools.product(*values)]

[update_params_dict_list[i]  for i in top_n_index]

[{'pretrain_num_epochs': 0, 'train_num_epochs': 100, 'dop': 0.1},
 {'pretrain_num_epochs': 0, 'train_num_epochs': 100, 'dop': 0.0}]

#### To test network pharamter difference among various [fold] or [pharam dict] 

In [4]:
#换个癌种和上面的BRCA对比看看

dir_path = "/home/wuys/pdr_code/results/test/code_adv_norm/cesc/Percent_sd/CESC/gdsc1_raw/all/"
# model_save_folder = "/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/pretrain_num_epochs_0_train_num_epochs_100_dop_0.0"
model_save_folder = os.path.join(dir_path,os.listdir(dir_path,)[0])
print(model_save_folder)
input_dim = 1676 #for CESC

# Pretrain model------
shared_encoder = MLP(input_dim=input_dim,
                         output_dim=128,
                         hidden_dims=[512, 256],
                         dop=0.1).to("cuda")

shared_decoder = MLP(input_dim=2 * 128,
                         output_dim=input_dim,
                         hidden_dims=[512, 256][::-1],
                         dop=0.1).to("cuda")
t_dsnae = DSNAE(shared_encoder=shared_encoder,
                    decoder=shared_decoder,
                    alpha=1,
                    input_dim=input_dim,
                    latent_dim=128,
                    hidden_dims=[512, 256],
                    dop=0.1).to("cuda")
t_dsnae.load_state_dict(torch.load(os.path.join(model_save_folder, 'a_t_dsnae.pt')))


# Finetune model------
fold_count = 0

ft_encoder = t_dsnae.shared_encoder
target_decoder = MLP(input_dim=128 + 300,
                         output_dim=1,
                         hidden_dims=[64, 32]).to("cuda")
target_classifier = EncoderDecoder_1(encoder=ft_encoder, decoder=target_decoder, 
                                        normalize_flag=True).to("cuda")
target_classifier.load_state_dict(
        torch.load(os.path.join(model_save_folder, f'target_classifier_{fold_count}.pt')))

# Print
net = target_classifier
for name,parameters in net.named_parameters():
        print(name)
        print(parameters)

/home/wuys/pdr_code/results/test/code_adv_norm/cesc/Percent_sd/CESC/gdsc1_raw/all/pretrain_num_epochs_300_train_num_epochs_1500_dop_0.1
encoder.module.0.0.weight
Parameter containing:
tensor([[ 1.0011e-02, -1.1521e-02, -1.4823e-03,  ..., -4.8845e-03,
          3.0600e-02, -3.1782e-02],
        [ 2.7823e-02, -1.4524e-02, -2.3905e-02,  ...,  1.4807e-02,
         -1.9136e-03,  3.6293e-03],
        [-3.3730e-02, -1.7981e-02, -2.1408e-02,  ...,  6.9462e-03,
          4.2371e-03,  8.5756e-03],
        ...,
        [-2.9410e-03, -1.5836e-02,  4.1856e-03,  ..., -1.9501e-02,
         -7.0839e-03, -8.8975e-03],
        [ 4.9397e-03, -5.7172e-05, -1.4410e-02,  ..., -2.0213e-02,
          1.0677e-02, -3.9534e-02],
        [ 9.8942e-03, -1.3032e-02,  4.9138e-03,  ...,  2.1050e-03,
          2.2211e-02,  8.1428e-03]], device='cuda:0', requires_grad=True)
encoder.module.0.0.bias
Parameter containing:
tensor([-3.2720e-02, -6.8551e-02, -7.4146e-03,  6.5703e-02,  1.2957e-02,
         2.9109e-02,  3.0667

In [6]:
dir_path = "/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/"
# model_save_folder = "/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/pretrain_num_epochs_0_train_num_epochs_100_dop_0.0"
model_save_folder = os.path.join(dir_path,os.listdir(dir_path,)[10])
print(model_save_folder)
t_dsnae.load_state_dict(torch.load(os.path.join(model_save_folder, 'a_t_dsnae.pt')))

# Finetune model------
fold_count = 1

ft_encoder = t_dsnae.shared_encoder
target_decoder = MLP(input_dim=128 + 300,
                         output_dim=1,
                         hidden_dims=[64, 32]).to("cuda")
target_classifier = EncoderDecoder_1(encoder=ft_encoder, decoder=target_decoder, 
                                        normalize_flag=True).to("cuda")
target_classifier.load_state_dict(
        torch.load(os.path.join(model_save_folder, f'target_classifier_{fold_count}.pt')))

# Print
net = target_classifier
for name,parameters in net.named_parameters():
        print(name)
        print(parameters)

/home/wuys/pdr_code/results/test/code_adv_norm/brca/Percent_sd/BRCA/gdsc1_raw/all/pretrain_num_epochs_100_train_num_epochs_200_dop_0.1
encoder.module.0.0.weight
Parameter containing:
tensor([[-0.0047, -0.0063,  0.0133,  ..., -0.0098,  0.0126,  0.0122],
        [-0.0077, -0.0184,  0.0067,  ...,  0.0056,  0.0071,  0.0124],
        [ 0.0325,  0.0139,  0.0219,  ..., -0.0149, -0.0172, -0.0136],
        ...,
        [ 0.0297,  0.0232,  0.0083,  ...,  0.0020,  0.0128,  0.0098],
        [ 0.0149, -0.0066, -0.0080,  ...,  0.0099,  0.0030,  0.0069],
        [ 0.0059, -0.0248, -0.0237,  ...,  0.0150,  0.0002,  0.0067]],
       device='cuda:0', requires_grad=True)
encoder.module.0.0.bias
Parameter containing:
tensor([-8.6638e-03,  1.0838e-02, -3.5873e-04, -2.8173e-02, -2.5636e-03,
        -2.6352e-02,  2.1164e-02,  2.7869e-02, -2.2580e-02, -2.5429e-02,
         8.5560e-03,  1.9868e-02,  4.4708e-02, -1.4426e-02, -4.7352e-02,
        -4.0588e-02,  3.7376e-03,  7.8794e-03,  6.1734e-03,  2.8757e-02,
 

In [7]:
os.path.join("/home/wuys/pdr_code","..","pdr_code")

'/home/wuys/pdr_code/../pdr_code'

: 